In [ ]:
PRO_ISRAELI = [
'israel', 'israeli', 'zionism', 'idf', 'jewish state', 'jerusalem', 'netanyahu', 'jewish', 'hebrew', 'zionist entity'
]
PRO_PALESTINIAN = [
    'hezbollah', 'sinwar', 'palestine', 'hamas', 'gaza', 'west bank', 'plo', 'abbas', 'intifada', 'nakba'
]

In [ ]:

import pandas as pd
import re
# Load the Excel file
def process_text():
  file_path = "posts_first_targil.xlsx"
  df = pd.read_excel(file_path, sheet_name=None)

  # Initialize the new DataFrame
  df_sentence = pd.DataFrame(columns=['sheet_name', 'id', 'sentence'])

  # Process each sheet in the Excel file
  for sheet_name, data in df.items():
      for index, row in data.iterrows():
          # Combine text based on the sheet
          if sheet_name == 'A-J':
              combined_text = " ".join(str(row[col]) for col in ['title', 'sub_title', 'Body Text'] if col in row and pd.notna(row[col]))
          else:
              combined_text = " ".join(str(row[col]) for col in ['title', 'Body Text'] if col in row and pd.notna(row[col]))

          # Split the combined text into sentences
          sentences = [sentence.strip() for sentence in re.split(r'[.!?]', combined_text) if sentence.strip()]

          # Append sentences to the new DataFrame
          for sentence in sentences:
              df_sentence = pd.concat([df_sentence, pd.DataFrame({ 'sheet_name': [sheet_name], 'id': [index], 'sentence': [sentence]})], ignore_index=True)
  df_sentence.head()
  return df_sentence

data = process_text()
print(len(data))


30125


In [ ]:
def extract_sentences(df, pro_israeli_words, pro_palestinian_words):
    extracted = []

    for idx, row in df.iterrows():
        doc_id = row['id']

        sentence = row['sentence'].strip().lower()
        sheet_name = row['sheet_name']
        # Check for pro-Israeli and pro-Palestinian keywords
        is_pro_israeli = any(word in sentence for word in pro_israeli_words)
        is_pro_palestinian = any(word in sentence for word in pro_palestinian_words)

        # Determine the type based on keywords
        if is_pro_israeli and not is_pro_palestinian:
            extracted.append((doc_id,sheet_name, sentence, 'pro-israeli'))
        elif is_pro_palestinian and not is_pro_israeli:
            extracted.append((doc_id,sheet_name, sentence, 'pro-palestinian'))

    return pd.DataFrame(extracted, columns=['id','sheet_name', 'sentence', 'lebal'])

data = extract_sentences(data,PRO_ISRAELI,PRO_PALESTINIAN)
print(len(data))

8909


In [ ]:
from tqdm import tqdm
import torch
import pandas as pd
from transformers import pipeline
# Run sentiment analysis using Hugging Face models

def sentiment_models():
    model_paths = {
        'model1': "cardiffnlp/twitter-roberta-base-sentiment-latest",
        'model2': "nlptown/bert-base-multilingual-uncased-sentiment",
        'model3': "cardiffnlp/twitter-xlm-roberta-base-sentiment",
        'model4': "siebert/sentiment-roberta-large-english",
        'model5': "lxyuan/distilbert-base-multilingual-cased-sentiments-student",
        'model6': "finiteautomata/bertweet-base-sentiment-analysis",
        'model7': "j-hartmann/sentiment-roberta-large-english-3-classes"
    }

    sent_models = {}
    for name, path in model_paths.items():
        sent_models[name] = pipeline("sentiment-analysis", model=path,device=0)
    return sent_models
models = sentiment_models()

In [ ]:
from transformers import AutoTokenizer

def map_sentiment_to_context(sentiment_label, score, sentence_context):
    """Map sentiment label to pro-Israeli/pro-Palestinian context."""

    # Normalize sentiment label
    sentiment = sentiment_label.lower()

    # Neutral sentiment handling
    if 'neutral' in sentiment:
        return 'NEUTRAL'

    # Map sentiment based on sentence context
    if sentence_context == 'pro-israeli':
        return 'POS' if sentiment in ['positive', 'pos'] else 'NEG'
    elif sentence_context == 'pro-palestinian':
        return 'POS' if sentiment in ['positive', 'pos'] else 'NEG'

    # Default to NEUTRAL if no match
    return 'NEUTRAL'


def evaluate_sentence_with_models(sentence, context_type, models, tokenizer_name="bert-base-uncased"):
    """
    Evaluate a sentence using multiple models, splitting it into chunks if necessary.

    Args:
        sentence (str): The input sentence to evaluate.
        context_type (str): The context type (e.g., 'pro-israeli', 'pro-palestinian').
        models (dict): A dictionary of model names and their instances.
        tokenizer_name (str): The tokenizer to use for token length calculation.

    Returns:
        dict: A dictionary of model results including scores and labels.
    """
    # Initialize tokenizer
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

    # Tokenize the sentence and check its length
    tokenized_sentence = tokenizer.encode(sentence, add_special_tokens=True)
    if len(tokenized_sentence) > 128:
        count += count
    if len(tokenized_sentence) <= 512:
        # Sentence fits within token limits
        return _evaluate_single_sentence(sentence, context_type, models)
    else:
        # Split the sentence into smaller chunks
        chunks = _split_into_chunks(sentence, tokenizer, max_length=512)
        aggregated_results = {}

        for chunk in chunks:
            chunk_results = _evaluate_single_sentence(chunk, context_type, models)
            for key, value in chunk_results.items():
                if key in aggregated_results:
                    aggregated_results[key].append(value)
                else:
                    aggregated_results[key] = [value]

        # Aggregate results across chunks
        return _aggregate_chunk_results(aggregated_results)


def _evaluate_single_sentence(sentence, context_type, models):
    """
    Evaluate a single sentence using multiple models.
    """
    evaluation_results = {}

    for model_name, model_instance in models.items():
        try:
            prediction = model_instance(sentence)[0]
            evaluation_results[f"{model_name}_score"] = prediction['score']
            evaluation_results[f"{model_name}_label"] = map_sentiment_to_context(
                prediction['label'], prediction['score'], context_type
            )
        except Exception as e:
            print(f"Error with model '{model_name}' for sentence: '{sentence[:50]}...' Error: {e}")
            evaluation_results[f"{model_name}_score"] = None
            evaluation_results[f"{model_name}_label"] = None

    return evaluation_results


def _split_into_chunks(sentence, tokenizer, max_length):
    """
    Split a sentence into chunks that fit within the token limit.

    Args:
        sentence (str): The sentence to split.
        tokenizer: The tokenizer instance.
        max_length (int): The maximum number of tokens per chunk.

    Returns:
        list: A list of sentence chunks.
    """
    words = sentence.split()
    chunks = []
    current_chunk = []

    for word in words:
        # Add word to the current chunk and check its token length
        current_chunk.append(word)
        if len(tokenizer.encode(" ".join(current_chunk), add_special_tokens=True)) > max_length:
            # Remove the last word and finalize the current chunk
            current_chunk.pop()
            chunks.append(" ".join(current_chunk))
            current_chunk = [word]

    # Add the final chunk
    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks


def _aggregate_chunk_results(results):
    """
    Aggregate results across multiple chunks by averaging scores and determining majority labels.
    """
    aggregated = {}

    for key, values in results.items():
        if '_score' in key:
            # Average scores
            aggregated[key] = sum(values) / len(values)
        elif '_label' in key:
            # Majority label
            aggregated[key] = max(set(values), key=values.count)

    return aggregated

def determine_majority_sentiment(row):
    """Determine the majority sentiment label across multiple models."""
    sentiment_labels = [value for key, value in row.items() if '_label' in key and value is not None]

    if not sentiment_labels:
        return 'UNKNOWN'

    # Determine majority label
    majority_label = max(set(sentiment_labels), key=sentiment_labels.count)
    return majority_label

In [ ]:
import os
os.environ['TORCH_USE_CUDA_DSA'] = '1'

import torch
# Process each sentence
results = []
for idx, row in tqdm(data.iterrows(), total=len(data)):
    result = {
        'newspaper': row['sheet_name'].split('_')[0],
        'article_id': row['id'],
        'sentence': row['sentence'],
        'type': row['lebal']
    }

    # Add model predictions
    result.update(evaluate_sentence_with_models(row['sentence'], row['lebal'], models))

    # Add to results
    results.append(result)
print(count)
# Create final DataFrame
output_df = pd.DataFrame(results)

# Add majority decision
output_df['majority_decision'] = output_df.apply(determine_majority_sentiment, axis=1)

# Calculate average score for majority decision
score_columns = [col for col in output_df.columns if '_score' in col]
output_df['avg_majority_score'] = output_df[score_columns].mean(axis=1)
# Save to Excel
output_df.to_excel('sentiment.xlsx', index=False)


In [ ]:
import os
os.environ['TORCH_USE_CUDA_DSA'] = '1'

import torch

# Now you can use CUDA and device-side assertions will be enabled.
print(torch.cuda.is_available())


True


In [ ]:
def determine_dominant_label(label_counts):
    """
    Determines the label with the highest count.
    If there is a tie, returns 'N/A'.
    """
    # Find the maximum count in the dictionary
    max_count = max(label_counts.values())

    # Identify all labels with the maximum count
    dominant_labels = [label for label, count in label_counts.items() if count == max_count]

    # Return the dominant label if there is only one, otherwise return 'N/A' for ties
    return dominant_labels[0] if len(dominant_labels) == 1 else 'N/A'

# Creating a list to store results for each paper
papers_results = []
for paper_id in output_df['newspaper'].unique():
    paper_data = output_df[output_df['newspaper'] == paper_id]

    articles_data = []

    for article_idx in paper_data['article_id'].unique():
        article_data = paper_data[paper_data['article_id'] == article_idx]

        label_counts = {'POS-I': 0, 'POS-P': 0, 'NUT': 0}
        score_sums = {'POS-I': 0, 'POS-P': 0, 'NUT': 0}

        for idx, row in article_data.iterrows():
            if (row['majority_decision'] == 'POS' and row['type'] == 'pro-israeli') or (row['majority_decision'] == 'NEG' and row['type'] == 'pro-palestinian'):
                label_counts['POS-I'] += 1
                score_sums['POS-I'] += row['avg_majority_score']
            elif (row['majority_decision'] == 'POS' and row['type'] == 'pro-palestinian') or (row['majority_decision'] == 'NEG' and row['type'] == 'pro-israeli'):
                label_counts['POS-P'] += 1
                score_sums['POS-P'] += row['avg_majority_score']
            elif row['majority_decision'] == 'NEUTRAL':
                label_counts['NUT'] += 1
                score_sums['NUT'] += row['avg_majority_score']

        dominant_label = determine_dominant_label(label_counts)
        articles_data.append({
            'article_index': article_idx,
            'dominant_label': dominant_label,
            'average_score': score_sums[dominant_label] / label_counts[dominant_label] if dominant_label != 'N/A' else 1,
        })

    papers_results.append((pd.DataFrame(articles_data), paper_id))

# Save to Excel with updated sheet names
with pd.ExcelWriter('updated_results.xlsx') as writer:
    for article_df, paper in papers_results:
        article_df.to_excel(writer, sheet_name=str(paper), index=False)


In [ ]:
for df, sheet_name in papers_results:
    pro_classification = df['dominant_label'].value_counts()
    pro_classification = pro_classification.to_dict()
    pro_classification['N/A'] = 0

    decided_pro = determine_dominant_label(pro_classification)
    score = df[df['dominant_label'] == decided_pro]['average_score'].mean() if decided_pro != 'N/A' else 'N/A'
    print(f'paper: {sheet_name} label: {decided_pro} score: {score}')

paper: A-J label: POS-P score: 0.7497990552337218
paper: BBC label: POS-P score: 0.7481500840890505
paper: J-P label: POS-P score: 0.7474796354869099
paper: NY-T label: POS-P score: 0.744396989673225
